### 필요한 라이브러리 설치

In [1]:
# !pip install networkx
# !pip install gensim
# !pip install pyLDAvis
# !pip install xlsxwriter
# !pip install nltk
# !pip install numpy

In [1]:
import re
import json
import pandas as pd
import xlsxwriter
import numpy as np

### 네이버 블로그 데이터 불러오기

In [2]:
blog_df = pd.read_csv('data/skm_blog.csv', encoding='utf-8-sig') # 네이버 블로그 csv파일 불러오기

In [3]:
blog_df = blog_df.sort_values(by='Post Date' ,ascending=True) # post Date를 기준으로 오름차순으로 정렬
blog_df['Post Date'] = [str(dates).replace('.0','') for dates in blog_df['Post Date']] # 분석을 위해서 날짜의 형식을 변경함. 

In [4]:
# 문서에 'Post Date'에서 빈칸을 nan (빈데이터)를 찾아주고 삭제해주는 과정 = 나중에 분석에 오류를 없애기 위해서
drop_index = blog_df['Post Date'][blog_df['Post Date']=='nan'].index
d = [d for d in drop_index] 
print(d, '\n') # 아래 결과 예) 751 번째 행의 날짜가 nan으로 표시 되어있음을 알 수 있음. 

blog_df.drop(d, axis=0, inplace=True) # nan 데이터가 있는 행을 삭제해줌.
print(blog_df.shape)

[362] 

(559, 7)


In [5]:
# 필요한 부분만 편집해서 가져오기. 
# 블로그 데이터 파일에서 블로그 제목인 'Title'과 블로그 내용인 'Description' 및 게시일인 'Post Date'에 해당하는 부분만 가져와 저장.
tm_post = blog_df.get('Title') + " " + blog_df.get('Description')
tm_post_date = blog_df.get('Post Date')

tm_post

292    [차트뉴스] SK매직, 식기세척기 시장 만년 1위 오죽하면 빨래건조기, 로봇 청소기...
521    동양매직에서 SK매직으로 이름이 바뀐 슈퍼 공기청정기...  동양매직에서 SK매직으...
96     SK매직 무선청소기 VCL-D200 개봉기 및 사용기 SK매직 무선청소기 VCL-D...
534    내돈주고 사서쓰는 유선진공청소기 SK매직 VCL-613 아기가 생긴 뒤로 내 머리카...
274    소형가전(무선청소기... ZB3233B /SK매직 EON-30AMD 다른 청소기를 ...
                             ...                        
103    삼성비스포크 렌탈 케어 SK매직 스페셜 렌탈 서비스 받은 후기! SK매직 렌탈서비스...
164    SK 매직 공기청정기 제로웨이스트 친환경가전 제대로네 공기청정기네요 SK매직 그린컬...
25     예쁜가전 SK매직 올클린 공기청정기로 집안공기까지 예쁘게~ 바로 예쁜가전 SK매직 ...
21     우리집 미세먼지는 SK매직 올클린 공기청정기 Green 242 환경까지 고려한 친환...
19     SK매직 공기청정기 Green 242 필수가전 인것 같아요 인 SK매직 올클린 공기...
Length: 559, dtype: object

In [8]:
# 필요한 부분만 편집해서 가져오기. 
# 블로그 데이터 파일에서 블로그 제목인 'title'과 블로그 내용인 'review' 및 게시일인 'date'에 해당하는 부분만 가져와 저장.
tm_post = shop_df.get('title') + " " + shop_df.get('review')
tm_post_date = shop_df.get('date')

tm_post

0      가스렌지 접지가 잘 되지 않아 1년에 한번씩은 수리를 받았어요작년에도 3만원 넘게 ...
1      직접 기존 고장난 제품을 제거하고 호스 끝을 가위로 잘라내고.. SK 매직 제품으로...
2      기존 스텐레스쿡탑 썼는데 아랫부분이 부식되서 녹가루 떨어져서 폭풍검색후 린나이. s...
3      여기저기 후기를 찾아보고 일주일 고민 끝에 주문했어요. 갑자기 사용하던 가스레인지가...
4      가스렌지 옛날거라 스마트센서도 없고 가스손잡이도 한참 잡고있어야 갑자기 불들어오고 ...
                             ...                        
995                              조아요 조아요 조아요 조아요 조아요 조아요
996    이거 좋네요. 기존 한샘꺼 썼는데 사이즈도 딱 맞고 불도 잘 켜지고 약불모드 기발한...
997      딱 맞았습니다. 감사합니다. 번창하세요.. 딱 맞았습니다. 감사합니다. 번창하세요..
998    가성비 좋고 배송기사도 친절하게 설치해 주시네요.. 가성비 좋고 배송기사도 친절하게...
999    배송도 빠르고 설치 가사님 도친절 하셨습니다 배송도 빠르고 설치 가사님 도친절 하셨습니다
Length: 1000, dtype: object

In [6]:
tm_post.head()  # 첫 5행 보여주기

292    [차트뉴스] SK매직, 식기세척기 시장 만년 1위 오죽하면 빨래건조기, 로봇 청소기...
521    동양매직에서 SK매직으로 이름이 바뀐 슈퍼 공기청정기...  동양매직에서 SK매직으...
96     SK매직 무선청소기 VCL-D200 개봉기 및 사용기 SK매직 무선청소기 VCL-D...
534    내돈주고 사서쓰는 유선진공청소기 SK매직 VCL-613 아기가 생긴 뒤로 내 머리카...
274    소형가전(무선청소기... ZB3233B /SK매직 EON-30AMD 다른 청소기를 ...
dtype: object

In [7]:
tm_post.tail()  # 마지막 5행 보여주기

103    삼성비스포크 렌탈 케어 SK매직 스페셜 렌탈 서비스 받은 후기! SK매직 렌탈서비스...
164    SK 매직 공기청정기 제로웨이스트 친환경가전 제대로네 공기청정기네요 SK매직 그린컬...
25     예쁜가전 SK매직 올클린 공기청정기로 집안공기까지 예쁘게~ 바로 예쁜가전 SK매직 ...
21     우리집 미세먼지는 SK매직 올클린 공기청정기 Green 242 환경까지 고려한 친환...
19     SK매직 공기청정기 Green 242 필수가전 인것 같아요 인 SK매직 올클린 공기...
dtype: object

In [9]:
tm_post_date.max()   # 최대 날짜 (가장 최근 데이터)

'20211101'

In [10]:
tm_post_date.min()  # 최소 날짜 (가장 오래된 데이터)

'20161202'

In [11]:
# tm_post 전처리

print(tm_post.isnull().values.any()) # 빈 행이 있는지 확인하기 ( 뒤에 에러가 날 수 있음 ) 
print(tm_post.isnull().sum()) # null 값이 총 몇개가 있는지 확인 

tm_post = tm_post.dropna(how = 'any') # Null 값이 존재하는 행을 제거

False
0


In [12]:
len(tm_post)

559

In [17]:
import re

def cleanText(datas): # 단어 전처리
    data1 = re.sub('[^A-Za-z0-9가-힣;,.?!]', ' ', datas)
    data1 = data1.replace(' 청소기', ' 청소기 ') #검색어 띄어쓰기 붙여주기
    data1 = data1.replace('청소기 ', ' 청소기 ') #검색어 띄어쓰기 붙여주기
    data1 = data1.replace('  ', ' ')
    #data1 = data1.replace('매직', 'SK매직')
    data2 = data1.split()
    return data1

In [18]:
tm_posts = []
for posts in tm_post:
    post = cleanText(posts)
    post1 = ' '.join(post)
    tm_posts.append(post)
    
tm_posts[:3]

[' 차트뉴스 SK매직, 식기세척기 시장 만년 1위 오죽하면 빨래건조기, 로봇 청소기 와 함께 신의 물건인 가전 3신기 로 꼽히겠는가. 물론... 나누면 SK매직이 절반 이상의 점유율을 차지하며 1위 자리를 굳게 지키고 있음을 확인할 수 있다. SK매직은... ',
 '동양매직에서 SK매직으로 이름이 바뀐 슈퍼 공기청정기... 동양매직에서 SK매직으로 이름이 바뀐 슈퍼 공기청정기 렌탈해서 사용하고 있어요. 안녕하세요.... 저희는 침구 청소기 받아서 사용하는데요. 엄청 좋더군요. 침구 청소 한번 하고 나면 청소기 에 먼지가 들어가... ',
 'SK매직 무선 청소기 VCL D200 개봉기 및 사용기 SK매직 무선 청소기 VCL D200 지금까지 유선청소기만 써 오다, 첫 무선 청소기 구입! 알아보고 알아보다 가성비 따져서 VCL D200을 구매하였습니다. 첫 무선 청소기 구입 기념으로 정성가득 포스팅을 해봅니다. 무선... ']

In [19]:
# 형태소 분석기 Mecab 불러오기

from konlpy.tag import Mecab

tagger = Mecab(dicpath='C:/mecab/mecab-ko-dic')

In [23]:
# 각자 데이터에 맞는 불용어를 추가하여 불용어 처리.
tm_stop_words = "게 점 자 수 후 저 쪽 음 은 는 위 이 가 배 거 츠 의 앞 강 글 애 미 번 것 등 더 를 좀 즉 인 옹 때 만 원 이때 개 일 기 시 트 드 기 탭"
tm_stop_words = tm_stop_words.split(' ')
print(tm_stop_words)

['게', '점', '자', '수', '후', '저', '쪽', '음', '은', '는', '위', '이', '가', '배', '거', '츠', '의', '앞', '강', '글', '애', '미', '번', '것', '등', '더', '를', '좀', '즉', '인', '옹', '때', '만', '원', '이때', '개', '일', '기', '시', '트', '드', '기', '탭']


In [24]:
# 불용어를 제외한 텍스트의 형태소 분석 수행

words = []
for post in tm_posts:
    words.extend(tagger.pos(post))
    
words[0:10]

[('차트', 'NNG'),
 ('뉴스', 'NNG'),
 ('SK', 'SL'),
 ('매직', 'NNG'),
 (',', 'SC'),
 ('식기세척기', 'NNG'),
 ('시장', 'NNG'),
 ('만년', 'NNG'),
 ('1', 'SN'),
 ('위', 'NNBC')]

In [25]:
tm_nouns = []   # 명사 추출하기
for post in tm_posts:
        if type(post) == str:
            for noun in tagger.nouns(post):
                if noun not in tm_stop_words:
                    tm_nouns.append(noun)
            
tm_nouns[0:10]

['차트', '뉴스', '매직', '식기세척기', '시장', '만년', '빨래', '건조기', '로봇', '청소기']

# 감성 분석(Sentiment Analysis)



## 한국어 감정분석

In [26]:
def data_list(wordname):

    with open('./data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:

        data = json.load(f)
    result = ['None','3']

    for i in range(0, len(data)):
        if data[i]['word'] == wordname:
            result.pop()
            result.pop()
            result.append(data[i]['word_root'])
            result.append(data[i]['polarity'])

    r_word = result[0]
    s_word = result[1]

    return [r_word, s_word]

#     print("-2:매우 부정, -1:부정, 0:중립 or Unkwon, 1:긍정, 2:매우 긍정")

In [27]:
tm_morphs = [] 
for post in tm_posts:
    posts = []
    if type(post) == str:
        for morphs in tagger.morphs(post):
            if morphs not in tm_stop_words:
                posts.append(morphs)
    posts = ' '.join(posts)
    tm_morphs.append(posts)

tm_morphs[0:3]

['차트 뉴스 SK 매직 , 식기세척기 시장 만년 1 오죽 하 면 빨래 건조기 , 로봇 청소기 와 함께 신 물건 가전 3 신기 로 꼽히 겠 는가 . 물론 . .. 나누 면 SK 매직 절반 이상 점유 율 을 차지 하 며 1 자리 굳 지키 고 있 을 확인 할 있 다 . SK 매직 . ..',
 '동양 매직 에서 SK 매직 으로 이름 바뀐 슈퍼 공기 청정기 . .. 동양 매직 에서 SK 매직 으로 이름 바뀐 슈퍼 공기 청정기 렌탈 해서 사용 하 고 있 어요 . 안녕 하 세요 . ... 저희 침구 청소기 받 아서 사용 하 는데요 . 엄청 좋 더군요 . 침구 청소 한번 하 고 나 면 청소기 에 먼지 들어가 . ..',
 'SK 매직 무선 청소기 VCL D 200 봉기 및 사용 SK 매직 무선 청소기 VCL D 200 지금 까지 유선 청소기 써 오 다 , 첫 무선 청소기 구입 ! 알아보 고 알아보 다 성비 따져서 VCL D 200 을 구매 하 였 습니다 . 첫 무선 청소기 구입 기념 으로 정성 가득 포스팅 을 해 봅니다 . 무선 . ..']

In [28]:
import json
from tqdm import tqdm

sentiment_firm=[]

for tm_words in tqdm(tm_morphs):
    sentiment_firm_line = []
    sentiment = 0
    count = 0
    
    for i in range(len(tm_words)):
        emotion_an = data_list(tm_words[i])
        if emotion_an[1] == '3':
            pass
        else:
            sentiment += int(emotion_an[1])
            count += 1
    try:
        sentiment_firm.append(sentiment/count)
    except:
        sentiment_firm.append(0)

100%|████████████████████████████████████████████████████████████████████████████████| 559/559 [28:08<00:00,  3.02s/it]


In [29]:
kor_sent=pd.DataFrame(sentiment_firm)

kor_sent

,0
0,0.000000
1,-0.250000
2,-0.250000
3,-2.000000
4,-0.500000
...,...
554,-0.333333
555,1.000000
556,1.333333
557,-0.800000


In [ ]:
# # 감성분석한 쇼핑후기 데이터 xlsx로 저장하기
# kor_sentiment = pd.ExcelWriter('./data/skm_shopping_sentiment.xlsx', engine='xlsxwriter')
# kor_sent.to_excel(kor_sentiment, sheet_name='Sheet1')
# kor_sentiment.close()

In [30]:
# 감성분석한 블로그 데이터 xlsx로 저장하기
kor_sentiment = pd.ExcelWriter('./data/skm_blog_sentiment.xlsx', engine='xlsxwriter')
kor_sent.to_excel(kor_sentiment, sheet_name='Sheet1')
kor_sentiment.close()